In [1]:
import pandas as pd
from pathlib import Path
#from updates_join_annotations import open_df_from_tsv
#import updates_get_average_scores_per_label as scores


In [6]:
# Read in file and set data type for two columns
filepath = './label_level_df.xlsx'
df = pd.read_excel(filepath, dtype={'tokens': list, 'annotators':set})

In [7]:
# Sort values and reset index to allow looping later
df.sort_values(by=['file_id', 'start_index', 'end_index', 'Label 1'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Label 1,Label 2,file_id,start_index,end_index,tokens,annotators,annotator_count
0,2503,48.0,54.0,view\,Patient,2503,48,54,"['Verteld', 'zelf', 'dat', 'hij', 'erg', 'prod...",{'katsburg'},1
1,2503,96.0,98.0,.B152: Stemming,NaN,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
2,2503,96.0,98.0,STM 1,NaN,2503,96,98,"['Mw', 'was', 'emotioneel']",{'vanderpas'},1
3,2503,97.0,98.0,.B152: Stemming,NaN,2503,97,98,"['was', 'emotioneel']",{'opsomer'},1
4,2503,98.0,98.0,.B152: Stemming,NaN,2503,98,98,['emotioneel'],"{'swartjes', 'avelli', 'vervaart', 'bos', 'mes...",6


In [48]:
from ast import literal_eval
#file_id = 2345
#label = 'STM0'
label_dict = dict()

# Loop through file_ids (change later)
for file_id in set(df['file_id']):
#for file_id in [2503]:
    # Create filtered df
    df_filtered = df[df['file_id'] == file_id]
    # Loop through labels and create further filtered df
    for label in set(df_filtered['Label 1']):
        
        counter = len(label_dict)+1
        
        print(file_id, label)
        
        df_label = df_filtered[df_filtered['Label 1'] == label]
        df_label.reset_index(inplace=True, drop=True)
        
        try:
            # Set start and end index values
            start_index = df_label['start_index'][0]
            end_index = df_label['end_index'][0]
            print('first annotation:', start_index, end_index)
        except:
            continue
        
        # If only one tag of label in this file
        if df_label.shape[0] == 1:
            counter = len(label_dict)+1
            label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
            continue
        
        # label through rest of df
        for index, row in df_label[1:].iterrows():
            # Note: df which it is looping over contains only data from one file from one label

            start_temp = row['start_index']
            end_temp = row['end_index']
            print('temp', start_temp, end_temp)

            current_range = range(start_index, end_index+1)

            # Since the df_label is sorted on start & end index, only a few cases need to be checked
            # If the temporary start vallue is in the range of the saved start value
            if start_temp in current_range:
                print('included')
                if end_temp not in current_range: # And the temp end value is not, then update the end value
                    print('updated')
                    end_index = end_temp
                # If the temp end value is also in the range then do nothing and move on.
            else:
                #print('else, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    if type(label_dict[counter]['annotators'])==str:
                        currentannos =literal_eval(label_dict[counter]['annotators'])
                    else:
                        currentannos =label_dict[counter]['annotators']
                    newannos = literal_eval(row['annotators'])
                    #print('Current annotators:',currentannos)
                    #print('Adding annotators:', newannos)
                    currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos
                    #print('Extended annotators:',label_dict[counter]['annotators'])

                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}

            # For last entry
            if index == len(df_label)-1:
                print('last, stored')
                if counter in label_dict:
                    label_dict[counter]['end_index'] = end_index
                    if type(label_dict[counter]['annotators'])==str:
                        currentannos =literal_eval(label_dict[counter]['annotators'])
                    else:
                        currentannos =label_dict[counter]['annotators']
                    newannos = literal_eval(row['annotators'])
                    #print('Current annotators:',currentannos)
                    #print('Adding annotators:', newannos)
                    currentannos.update(newannos)
                    label_dict[counter]['annotators'] = currentannos
                    #print('Extended annotators:',label_dict[counter]['annotators'])
                   
                else: 
                    label_dict[counter] = {'file_id': file_id,
                                           'start_index': start_index, 
                                           'end_index': end_index,
                                           'label': label,
                                           'tokens': row['tokens'],
                                           'annotators': row['annotators']}
    
    
                    
#label_dict   

4100 view\
first annotation: 263 282
temp 510 513
last, stored
4100 FAC 0
first annotation: 556 556
4100 FAC 2
first annotation: 517 519
temp 517 520
included
updated
last, stored
4100 .B455: Inspanningstolerantie
first annotation: 157 158
temp 158 158
included
last, stored
4100 type\
first annotation: 14 148
temp 16 49
included
temp 18 40
included
temp 45 48
included
temp 50 82
included
temp 55 65
included
temp 67 82
included
temp 84 125
included
temp 88 115
included
temp 121 130
included
temp 126 148
included
temp 132 148
included
temp 507 507
temp 556 558
last, stored
4100 INS 3
first annotation: 157 158
4100 FAC 1
first annotation: 515 519
temp 515 520
included
updated
temp 515 524
included
updated
temp 517 519
included
last, stored
4100 .D450: Lopen en zich verplaatsen
first annotation: 507 507
temp 515 520
temp 520 520
temp 557 558
temp 558 558
last, stored
4103 .B152: Stemming
first annotation: 338 341
temp 341 607
included
updated
temp 395 395
included
temp 533 538
included
tem

In [52]:
df_soft = pd.DataFrame.from_dict(label_dict, orient='index')
df_soft.to_excel('softboundary_output_file.xlsx')

In [ ]:
# file_id = 1234
# label = 'Stemming'
# label_dict = {label: dict()}
# # Set impossible start and end index values
# start_index = -2
# end_index = -1

# for index, row in df_label.iterrows():
#     # Note: df which it is looping over contains only data from one file from one label
#     #counter = len(label_dict[label])+1
    
#     start_temp = row['start_index']
#     end_temp = row['end_index']
#     print('temp', start_temp, end_temp)
    
#     current_range = range(start_index, end_index+1)
    
#     # If temp start and end are in currently stored start-end range, change nothing
# #     if start_temp in current_range and end_temp in current_range:
# #         continue
#     # If temp start in the current range (there is overlap), and end temp is not in the current range, update the end index
#     if start_temp in current_range and not end_temp in current_range:
#         print('a')
#         end_index = end_temp
#     # Elif temp end in the current range (there is overlap), and start temp is not in the current range, update the start index
#     elif end_temp in current_range and not start_temp in current_range:
#         print('b')
#         start_index = start_temp
#     # Elif if the temp range encompasses the stored/current range, update both start and end indices
#     elif start_index in range(start_temp, end_temp+1) and end_index in range(start_temp, end_temp+1):
#         print('c')
#         start_index = start_temp
#         end_index = end_temp
        
#     elif all([not start_temp in current_range,
#               not end_temp in current_range,
#               start_index >= 0,  # As to not save initial values
#               end_index >=0]): 
#         # If new start and end indices are disjoint from stored indices, then save and restart
#         print('d')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#         start_index = start_temp
#         end_index = end_temp
        
#     if index == len(df_label)-1:
#         print('yes')
#         counter = len(label_dict[label])+1
#         label_dict[label][counter] = (start_index, end_index)
#     print('stored', start_index, end_index)